### Documentation References:
* Use google-cloud-sdk version 1.20.0
* From the GAPIC client - [Endpoint Service Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.endpoint_service.EndpointServiceClient)
* [Update Endpoint](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.endpoint_service.EndpointServiceClient#google_cloud_aiplatform_v1_services_endpoint_service_EndpointServiceClient_update_endpoint)
* [Endpoint Class](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.Endpoint)
* [Request-Response-Logging-Config](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.PredictRequestResponseLoggingConfig)
* [BigQueryDestination](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.BigQueryDestination)


### Update via the GAPIC SDK
The following cell shows an example of enabling request response logging via the `aiplatform_v1` GAPIC (Google API Compiler) SDK

In [79]:
from google.cloud import aiplatform_v1
from google.cloud.aiplatform_v1.types import Endpoint,PredictRequestResponseLoggingConfig, BigQueryDestination
from google.protobuf import field_mask_pb2 as field_mask

PROJECT='gcp-ml-sandbox'
REGION='us-central1'
ENDPOINT_ID='8031541014764191744'

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Logging destination
BQ_DATASET="scratch"
BQ_TABLE=f"endpoint_{ENDPOINT_ID}_predict_req_resp_log"


def enable_predict_request_response_logging():
    
    client_options = {"api_endpoint": API_ENDPOINT}
    endpoint_name = f"projects/{PROJECT}/locations/{REGION}/endpoints/{ENDPOINT_ID}"

    
    
    endpoint = aiplatform_v1.Endpoint(
        {"name":endpoint_name,
         "predict_request_response_logging_config" : 
             PredictRequestResponseLoggingConfig(
                 {
                    "enabled":True,
                    "sampling_rate":.3, # Float value between 0 and 1
                    "bigquery_destination": 
                         BigQueryDestination({"output_uri":f"bq://{PROJECT}.{BQ_DATASET}.{BQ_TABLE}"})
                 }
             )
        }
    )
    update_mask = field_mask.FieldMask(paths=['predict_request_response_logging_config'])
    
    

    request = aiplatform_v1.UpdateEndpointRequest(
        endpoint=endpoint,
        update_mask=update_mask
    )
    
    # Make the request
    client = aiplatform_v1.EndpointServiceClient(client_options=client_options)
    response =  client.update_endpoint(request=request)

    # Handle the response
    print(f"Endpoint: {response.name}\nPredictRequestResponseLoggingConfig: {response.predict_request_response_logging_config}")


enable_predict_request_response_logging()

Endpoint: projects/357746845324/locations/us-central1/endpoints/8031541014764191744
PredictRequestResponseLoggingConfig: enabled: true
sampling_rate: 0.3
bigquery_destination {
  output_uri: "bq://gcp-ml-sandbox.scratch.endpoint_8031541014764191744_predict_req_resp_log"
}



Double checking the output by running `get_endpoint`

In [76]:
from google.cloud import aiplatform_v1
from google.cloud.aiplatform_v1.types import Endpoint,PredictRequestResponseLoggingConfig, BigQueryDestination
from google.protobuf import field_mask_pb2

PROJECT='gcp-ml-sandbox'
REGION='us-central1'
ENDPOINT_ID='8031541014764191744'

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

def get_endpoint():
    # Create a client
    client_options = {"api_endpoint": API_ENDPOINT}
    endpoint_name = f"projects/{PROJECT}/locations/{REGION}/endpoints/{ENDPOINT_ID}"
    
    client = aiplatform_v1.EndpointServiceClient(client_options=client_options)

    # Initialize request argument(s)
    request = aiplatform_v1.GetEndpointRequest(
        name=endpoint_name,
    )

    # Make the request
    response = client.get_endpoint(request=request)

    # Handle the response
    print(f"Endpoint: {response.name}\nPredictRequestResponseLoggingConfig: {response.predict_request_response_logging_config}")

get_endpoint()

Endpoint: projects/357746845324/locations/us-central1/endpoints/8031541014764191744
PredictRequestResponseLoggingConfig: enabled: true
sampling_rate: 0.3
bigquery_destination {
  output_uri: "bq://gcp-ml-sandbox.scratch.endpoint_8031541014764191744_predict_req_resp_log"
}



In [88]:
ENDPOINT_URI = "https://us-central1-aiplatform.googleapis.com/v1/projects/357746845324/locations/us-central1/endpoints/8031541014764191744"
endpoint_name = f"projects/{ENDPOINT_URI.split ('/projects')[1]}"
endpoint_name


'projects//357746845324/locations/us-central1/endpoints/8031541014764191744'

### Custom Component and Pipeline Definition
In the following cell, a python-based custom component is used to wrap the SDK call and handle pipeline artifacts for input and output
The pipeline definition also uses the [dsl.importer](https://www.kubeflow.org/docs/components/pipelines/v1/sdk-v2/importer-component/) to bring in a reference to the Artifact URI from a prior pipeline run

In [95]:
from kfp.v2 import compiler, dsl
from google_cloud_pipeline_components.types import artifact_types


BASE_DIR = "gs://gcp-ml-sandbox-scratch/endpoint-patch"
PIPELINE_ROOT = f"{BASE_DIR}/pipeline-root"

################################################################
# Simple Custom Component to Patch a Vertex Endpoint using the 
# aiplatform_v1 GAPIC SDK
# to enable Vertex request-response logging

################################################################

@dsl.component(base_image='python:3.9-slim',
           packages_to_install=['google-cloud-aiplatform'])

def enable_request_response_logging(
    input_endpoint: dsl.Input[dsl.Artifact],
    updated_endpoint: dsl.Output[dsl.Artifact],
    bq_log_destination: str,
    sampling_rate: float,
    project: str,
    region: str="us-central1",
):
    from google.cloud import aiplatform_v1
    from google.cloud.aiplatform_v1.types import Endpoint,PredictRequestResponseLoggingConfig, BigQueryDestination
    from google.protobuf import field_mask_pb2 as field_mask
    
    import copy
    
    # Grabbing a path to the Endpoint URI from the google.VertexEndpoint artifact
    endpoint_name = f"projects{input_endpoint.uri.split ('/projects')[1]}"
    
    endpoint = aiplatform_v1.Endpoint(
        {"name":endpoint_name,
         "predict_request_response_logging_config" : 
             PredictRequestResponseLoggingConfig(
                 {
                    "enabled":True,
                    "sampling_rate":sampling_rate, # Float value between 0 and 1
                    "bigquery_destination": 
                         BigQueryDestination({"output_uri":bq_log_destination})
                 }
             )
        }
    )
    update_mask = field_mask.FieldMask(paths=['predict_request_response_logging_config'])

    request = aiplatform_v1.UpdateEndpointRequest(
        endpoint=endpoint,
        update_mask=update_mask
    )
    
    # Make the request
    api_endpoint = "{}-aiplatform.googleapis.com".format(region)
    client_options = {"api_endpoint": api_endpoint}
    
    print(f"API Endpoint: {api_endpoint}")
    
    client = aiplatform_v1.EndpointServiceClient(client_options=client_options)
    response =  client.update_endpoint(request=request)

    # Handle the response
    print(f"Endpoint: {response.name}\nPredictRequestResponseLoggingConfig: {endpoint.predict_request_response_logging_config}")
    
    #Copying the input artifact object to the output artifact
    updated_endpoint = copy.copy(input_endpoint)
    
    

####################################################################
# Pipeline Definition
####################################################################
    

@dsl.pipeline(pipeline_root=PIPELINE_ROOT, name="endpoint-patch-example")
def endpoint_pipeline(
    bq_log_destination: str,
    project: str,
    region: str,
    endpoint_uri: str,
    sampling_rate: float
):
    endpoint_uri=endpoint_uri
    
    endpoint_task = dsl.importer(
        artifact_uri=endpoint_uri,
        artifact_class=artifact_types.VertexEndpoint,
      ).set_display_name('Import endpoint')
    
    _ = enable_request_response_logging(
            input_endpoint=endpoint_task.output,
            bq_log_destination=bq_log_destination,
            sampling_rate=sampling_rate,
            project=project,
            region=region
        ).set_display_name("Patching Endpoint")
    



### Run Pipeline and patch endpoint
The following cell shows compiling and running the pipeline, passing in an existing artifact URI for the endpoint as a parameter.  If not using the `dsl.Importer`, you can pass in a VertexEndpoint to the custom component created above.

In [96]:
#################################################
### Setup Pipeline parameters for this example
### and run pipeline

import google.cloud.aiplatform as aip

from kfp.v2 import compiler
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
PIPELINE_SPEC='endpoint_pipeline.json'
BASE_DIR = "gs://gcp-ml-sandbox-scratch/endpoint-patch"
PIPELINE_ROOT = f"{BASE_DIR}/pipeline-root"

PROJECT='gcp-ml-sandbox'
REGION='us-central1'
ENDPOINT_URI = "https://us-central1-aiplatform.googleapis.com/v1/projects/357746845324/locations/us-central1/endpoints/8031541014764191744"

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Logging destination
BQ_DATASET="scratch"
BQ_TABLE=f"endpoint_{ENDPOINT_ID}_predict_req_resp_log"
BQ_LOG_DESTINATION=f"bq://{PROJECT}.{BQ_DATASET}.{BQ_TABLE}"

SAMPLING_RATE=.35

compiler.Compiler().compile(pipeline_func=endpoint_pipeline,
        package_path=PIPELINE_SPEC)

job = aip.PipelineJob(
    display_name=f"pipeline_endpoint_patch_{TIMESTAMP}",
    template_path=PIPELINE_SPEC,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "project": PROJECT,
        "region": REGION,
        "bq_log_destination": BQ_LOG_DESTINATION,
        "endpoint_uri": ENDPOINT_URI,
        "sampling_rate":SAMPLING_RATE
    }
)

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/357746845324/locations/us-central1/pipelineJobs/endpoint-patch-example-20221222164933
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/357746845324/locations/us-central1/pipelineJobs/endpoint-patch-example-20221222164933')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/endpoint-patch-example-20221222164933?project=357746845324
